In [ ]:
# default_exp hirise

# HiRISE tools

> A collection of HiRISE related tools.

Currently depending on my older `pyrise` module, which will be more cleanly implemented here over time.

In [ ]:
# export

import warnings

import hvplot.xarray
import rasterio
import xarray as xr
from dask import compute, delayed
from planetarypy.config import config
from planetarypy.pds.apps import find_indexes, get_index
from planetarypy.utils import url_retrieve
from pyrise import downloads as dl
from pyrise import products as prod
from yarl import URL

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [ ]:
# export
storage_root = config.storage_root / "mro/hirise"
baseurl = URL("https://hirise-pds.lpl.arizona.edu/download/PDS")
rdrindex = get_index("mro.hirise.indexes.rdr")

In [ ]:
# export


class RGB_NOMAP:
    def __init__(self, obsid):
        self.obsid = obsid

    @property
    def product_id(self):
        return self.obsid + "_COLOR"

    @property
    def filename(self):
        return self.obsid + ".JP2"

    @property
    def pid(self):
        pid = prod.PRODUCT_ID(self.obsid)
        pid.kind = "RGB"
        return pid

    @property
    def meta(self):
        s = rdrindex.query("PRODUCT_ID == @self.product_id").squeeze()
        s.index = s.index.str.lower()
        return s

    # several things in the PDS path either have changed or I did it wrong in pyrise
    @property
    def nomap_jp2_path(self):
        return "EXTRAS/" + self.pid.nomap_jp2_path.replace("/EXTRAS", "")

    @property
    def url(self):
        return baseurl / self.nomap_jp2_path

    @property
    def local_path(self):
        return storage_root / self.nomap_jp2_path

    def download(self, overwrite=False):
        self.local_path.parent.mkdir(parents=True, exist_ok=True)
        if self.local_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        url_retrieve(self.url, self.local_path)

    def read(self):
        self.da = xr.open_rasterio(self.local_path, chunks=(1, 2024, 2024))
        return self.da

    def plot_da(self):
        return self.da.hvplot.image(
            x="y",
            y="x",
            rasterize=True,
            widget_location="top_left",
            cmap="gray",
        )

In [ ]:
obsid = "ESP_011296_0975"

In [ ]:
rgb = RGB_NOMAP(obsid)

In [ ]:
da = rgb.read()
da

/home/maye/miniconda3/envs/py38/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


<xarray.DataArray (band: 3, y: 50000, x: 1012)>
dask.array<chunksize=(1, 2024, 1012), meta=np.ndarray>
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 5e+04 5e+04 5e+04 5e+04 5e+04
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 1.01e+03 1.01e+03 1.012e+03
Attributes:
    transform:   (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:         (1.0, -1.0)
    is_tiled:    0
    nodatavals:  (nan, nan, nan)
    scales:      (1.0, 1.0, 1.0)
    offsets:     (0.0, 0.0, 0.0)

In [ ]:
rgb.plot_da()

Column
    [0] Row
        [0] WidgetBox
            [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='band', options=OrderedDict([('1', ...]), value=1, width=250)
        [1] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top_left')

In [ ]:
rgb.local_path

PosixPath('/home/maye/big_drive/planetary_data/mro/hirise/EXTRAS/RDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_011296_0975_RGB.NOMAP.JP2')

In [ ]:
rgb.download()

File exists. Use `overwrite=True` to download fresh.


In [ ]:
rgb.meta

volume_id                                                              MROHR_0001
file_name_specification         RDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_...
instrument_host_id                                                            MRO
instrument_id                                                              HIRISE
observation_id                                                    ESP_011296_0975
product_id                                                  ESP_011296_0975_COLOR
product_version_id                                                              2
target_name                                                                  MARS
orbit_number                                                                11296
mission_phase_name                                         Extended Science Phase
rationale_desc                                                   Fans in clusters
observation_start_time                                        2008-12-23 16:15:26
observation_star

In [ ]:
rgb.url

URL('https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_011296_0975_RGB.NOMAP.JP2')

In [ ]:
rgb.nomap_jp2_path

'EXTRAS/RDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_011296_0975_RGB.NOMAP.JP2'

In [ ]:
# export


class RGB_NOMAPCollection:
    """Class to deal with a set of RGB_NOMAP products."""

    def __init__(self, obsids):
        self.obsids = obsids

    def get_urls(self):
        """Get URLs for list of obsids.

        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            urls.append(rgb.url)
        self.urls = urls
        return urls

    @property
    def local_paths(self):
        paths = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            paths.append(rgb.local_path)
        return paths

    def download_collection(self):
        lazys = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            lazys.append(delayed(rgb.download)())
        print("Launching parallel download...")
        compute(*lazys)
        print("Done.")